# Compute similarities of previous study for FGVC data.

Reproduce the results of [Measuring semantic similarity between concepts in visual domain](http://ieeexplore.ieee.org/document/4665152/).

In [1]:
%matplotlib inline

from scipy.misc import imread
import numpy as np
from skimage import color, filters, img_as_float
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import warnings
warnings.filterwarnings('ignore')

Define functions for getting tiles in an image and normalizing an image.

In [2]:
def get_tile(tile_row, tile_col, img, img_gray, img_binary):
    tile_h = int(img.shape[0]/5)
    tile_w = int(img.shape[1]/5)
    y0, y1, x0, x1 = tile_row*tile_h, ((tile_row+1)*tile_h), tile_col*tile_w, (tile_col+1)*tile_w
    tile = img[y0:y1, x0:x1, :]
    tile_gray = img_gray[y0:y1, x0:x1]
    tile_binary = img_binary[y0:y1, x0:x1]
    return tile, tile_gray, tile_binary

In [3]:
from scipy.misc import imresize
MAX_SIZE = 225

def im_normalized_read(fname, size_limit = MAX_SIZE):
    from skimage.transform import resize
    img =  imread(fname)
    max_len = np.max(img.shape)
    if max_len <= size_limit:
        return img_as_float(img)
    
    ratio = size_limit/max_len
    new_img = imresize(img, size=ratio, interp='bicubic')
    return img_as_float(new_img)

## Feature extraction

Here we treat four types of fetures: coloer moment, DOOG filter, orientation filter, shape features.

### Color moment

In [4]:
def calc_moment3(arr, moment1):
    cubed = np.mean((arr-moment1)**3)
    return math.pow(abs(cubed),1/3) * (1,-1)[cubed<0]

In [5]:
def calc_moments(arr):
    moment1 = np.mean(arr)
    moment2 = np.mean((arr-moment1)**2)**(1/2)
    moment3 = calc_moment3(arr, moment1)
    return moment1, moment2, moment3

In [6]:
def calc_4_rgb_moments(rgbarr):
    r1, r2, r3 = calc_moments(rgbarr[:, :, 0])
    g1, g2, g3 = calc_moments(rgbarr[:, :, 1])
    b1, b2, b3 = calc_moments(rgbarr[:, :, 2])
    gray = color.rgb2gray(rgbarr)
    gr1, gr2, gr3 = calc_moments(gray)
    return [r1, r2, r3, g1, g2, g3, b1, b2, b3, gr1, gr2, gr3]

### DOOG filter

Based on Figure 2 in [Preattentive texture discrimination with early vision mechanisms](https://www.ncbi.nlm.nih.gov/pubmed/2338600).

In [7]:
def G(y0, sigmax, sigmay, x, y):
    # The definition of G seems wrong, I refer original Yang's paper.
    # return (1/(2*math.pi* sigmax*sigmay)) *np.exp(- (x**2) + (y- y0/sigmay)**2)
    dimx = len(x)
    dimy = len(y)
    return (1/(2*math.pi* sigmax*sigmay)) *np.exp(- (1/2) * (((x/sigmax)**2).reshape(1, dimx) + (((y- y0)/sigmay)**2).reshape(dimy, 1)))

In [8]:
def DOOG2(x, y, sigma, r=3):
    sigmay = sigma
    sigmax = r*sigma
    ya = sigma
    yc = -sigma
    return -G(ya, sigmax, sigmay, x, y) + 2*G(0, sigmax, sigmay, x, y) - G(yc, sigmax, sigmay, x, y)

In [9]:
def DOOG2_weight(sigma, r=3, truncate=4):
    # similar filter size logic as filters.gaussian of scipy.
    lw = int(truncate * sigma + 0.5)
    return DOOG2(np.arange(-lw, lw+1), np.arange(-lw, lw+1), sigma, r)

In [10]:
# https://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.ndimage.filters.convolve.html
from scipy.ndimage.filters import convolve

In [11]:
def kernel2feature(img, weights):
    return np.mean([np.mean(convolve(img[:, :, i], weights)) for i in range(3)])

In [12]:
# use global variable for speed
# use sigma of 1, 2, 4, 8

KERNELS = [DOOG2_weight(i) for i in [1, 2, 4, 8]]

In [13]:
def tile2doog_features(tile):
    # to make scale similar
    return [1000*kernel2feature(tile, kernel) for kernel in KERNELS]

### Orientation feature

In [14]:
from scipy.stats import norm

In [15]:
def DOG(y, sigma1=0.5, sigma2=1.5):
    fy1 = norm.pdf(y, scale=sigma1)
    fy2 = norm.pdf(y, scale=sigma2)
    dog = fy1 - fy2
    return dog

In [16]:
def calc_f1(x, y, sigma):
    fx = norm.pdf(x, scale=sigma).reshape(1, len(x))
    fy = DOG(y).reshape(len(y), 1)
    # should access f1[y][x]
    return fx*fy

In [17]:
from scipy.signal import hilbert

In [18]:
def calc_f1f2(x, y, sigma):
    f1 = calc_f1(x, y, sigma)
    f2 = np.imag(hilbert(f1))
    return f1, f2

In [19]:
def rotation_indices(theta, shape):
    """return ([y1, y2, y3, ...], [x1, x2, x3, ...]). Use for numpy indices"""
    y_lim, x_lim = shape
    cos = math.cos(theta)
    sin = math.sin(theta)
    pairs = [(min(y_lim-1, max(0, int(x*sin + y*cos))),
              min(x_lim-1, max(0, int(x*cos - y*sin))))
            for y in range(y_lim)
            for x in range(x_lim)]
    return list(zip(*pairs))

Compute weights

In [20]:
def orientation_weights(theta, sigma, truncate=4):
    # similar filter size logic as filters.gaussian of scipy.
    lw = int(truncate * sigma + 0.5)
    
    x = np.arange(-lw, lw+1)
    y = np.arange(-lw, lw+1)
    f1, f2 = calc_f1f2(x, y, sigma)
    inds = rotation_indices(theta, f1.shape)
    f1r = f1[inds].reshape(f1.shape)
    f2r = f2[inds].reshape(f2.shape)
    return f1r, f2r

In [21]:
def orientation_energy(img, theta, sigma=1.5, truncate=4):
    f1, f2 = orientation_weights(theta, sigma)
    ex2 = convolve(img, f1)**2
    ey2 = convolve(img, f2)**2
    # make scale similar
    return 1000*np.mean(ex2+ey2)

In [22]:
def orientation_12_energy(gray_img):
    return [orientation_energy(gray_img, i*(2*math.pi)/12) for i in range(12)]

### Shape features

In [23]:
from skimage.filters import threshold_otsu

In [24]:
def gray2binary(gray_img):
    threds = threshold_otsu(gray_img)
    return gray_img <= threds

In [25]:
import skimage

#### Perimeter, area

In [26]:
# Paper says the ratio of the area to the perimeter squared, but inverse is much more common, which is compactness.
# standard definition divide this value with 4pi, but it's not important for our case.

def compactness(img_binary):
    l = skimage.measure.perimeter(img_binary)
    area = img_binary.sum()
    if area == 0:
        return 1, area
    return (l**2)/area, area

#### Moment of inertia

In [27]:
def calc_moment_of_inertia(gray_img):
    # cr, cc = ``M[1, 0] / M[0, 0]``, ``M[0, 1] / M[0, 0]`
    M1 = skimage.measure.moments(gray_img, order=1)
    cr = M1[1, 0]/M1[0, 0]
    cc = M1[0, 1]/M1[0, 0]
    mu2 = skimage.measure.moments_central(gray_img, cr, cc, order=2)
    # paper says shape feature is 4. So I use mu2_20 and mu2_02 as a separate features
    # return (mu2[2, 0]+mu2[0, 2])/mu2[0, 0]
    M0sq = mu2[0, 0]**2
    return mu2[2, 0]/M0sq, mu2[0, 2]/M0sq

In [28]:
from skimage.morphology import convex_hull_image
def ratio_convex_hull(img_bainary, area):
    hull_area = convex_hull_image(img_bainary).sum()
    return area/hull_area

#### Combine all shape features

In [29]:
def shape_features(img_gray, img_binary):
    comp, area = compactness(img_binary)
    mom = calc_moment_of_inertia(img_gray)
    if(img_binary.any()):
        ratio3 = ratio_convex_hull(img_binary, area)
    else:
        ratio3 = 1
    return [comp, mom[0], mom[1], ratio3]

### Combine all of features for a given tile

Define a function that combine all of featuers defined above.

In [30]:
def tile2features(tile, tile_gray, tile_binary):
    rgb_moments12 = calc_4_rgb_moments(tile)
    doog4 = tile2doog_features(tile)
    orientation12 = orientation_12_energy(tile_gray)
    shape4 = shape_features(tile_gray, tile_binary)
    return [elem for arr in [rgb_moments12, doog4, orientation12, shape4]
            for elem in arr]

### Define functions for extracting features for a given image.

A given image is divided into patches and features are extracted from each patch.  
Finally we concatenate those features into one feature vector to express characteristics of a given image.

In [31]:
def get_tile_features(tile_row, tile_col, img, img_gray, img_binary):
    tile, tile_gray, tile_binary = get_tile(tile_row, tile_col, img, img_gray, img_binary)
    return tile2features(tile, tile_gray, tile_binary)

In [32]:
def img2features(img):
    img_gray = color.rgb2gray(img)
    img_binary = gray2binary(img_gray)
    featuresList = [get_tile_features(row, col, img, img_gray, img_binary) for row in range(5)
                    for col in range(5)]
    return [f for fs in featuresList
               for f in fs]

## Extract features and save them into dataframe

In [33]:
from models.modelutils import dir2filedict_sorted

Using TensorFlow backend.


In [34]:
fdict1 = dir2filedict_sorted("data_fgvc/train")
fdict2 = dir2filedict_sorted("data_fgvc/valid")
fdict3 = dir2filedict_sorted("data_fgvc/test")

In [35]:
files = sorted(f for fdict in [fdict1, fdict2, fdict3] for key in fdict.keys() for f in fdict[key])

In [36]:
len(files)

10000

In [47]:
# machine_id=0
machine_id=1
# machine_id=2
# machine_id=3

In [48]:
target_files = files[(machine_id*2500):(machine_id*2500+2500)]

In [50]:
import pandas as pd

In [51]:
failed = []
flist = []

In [52]:
import tqdm

In [54]:
%%time

for file in tqdm.tqdm(target_files):
    try:
        current = file
        features = img2features(im_normalized_read(file))
        flist.append(features)
    except:
        print("fail {}".format(file))
        failed.append(file)

100%|██████████| 5/5 [01:06<00:00, 13.35s/it]

CPU times: user 57.9 s, sys: 8.44 s, total: 1min 6s
Wall time: 1min 6s


In [55]:
print( "Total number of images: {}".format(len(flist)) )
print( "The number of failed images from where features cannot be extracted: {}".format(len(failed)) )

Total number of images: 5
The number of failed images from where features cannot be extracted: 0


Check images from which features cannot be extracted.

In [56]:
failed

[]

Pick up images from which features can be successfully extracted,

In [57]:
files2 = [f for f in target_files if f not in failed]
len(files2)

5

In [58]:
df = pd.DataFrame(flist)
df.shape

(5, 800)

In [59]:
df['filepath'] = files2

Save the extracted features as pickle.

In [60]:
df.to_pickle("results/features_{}_fgvc.dat".format(machine_id))

## Load extracted features and estimate GMM

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
df = pd.read_pickle("results/features.dat")
df.shape

(11776, 801)

In [3]:
def path_to_cat(fpath):
    return fpath.split("/")[1]

In [4]:
path_to_cat(df['filepath'][5000])

'clouds'

In [5]:
cats = sorted(list(set(map(path_to_cat, df['filepath']))))

In [6]:
cats

['bay',
 'beach',
 'birds',
 'boeing',
 'buildings',
 'city',
 'clouds',
 'f-16',
 'face',
 'helicopter',
 'mountain',
 'ocean',
 'ships',
 'sky',
 'sunrise',
 'sunset']

In [7]:
df['category'] = list(map(path_to_cat, df['filepath']))

### For empty tile, moment will be na; fill na as 0 to avoid errors

In [8]:
df_fixed = df.fillna(0)

### Fit GMM for each class

We use the same 25 components GMM as that of the previsou study.

In [9]:
from sklearn.mixture import GaussianMixture

In [10]:
num_components = 25

In [11]:
def cat_to_model(catindex):
    onecat_df = df_fixed[df_fixed['category'] == cats[catindex]]
    onecat_fs = onecat_df.iloc[:, 0:800].as_matrix()
    
    # Section 4 in the paper denotes they assume N(mu, sigma) type.
    gmm = GaussianMixture(num_components, covariance_type = "diag")
    gmm.fit(onecat_fs)
    return gmm

In [12]:
%%time
models = [cat_to_model(ind) for ind in range(len(cats))]

CPU times: user 22.8 s, sys: 16.8 s, total: 39.6 s
Wall time: 13.3 s


Collect (means, variances) and weights for each class.

In [13]:
gmm_stats = [(mod.means_, mod.covariances_) for mod in models]

In [14]:
gmm_weights = [mod.weights_ for mod in models]

## Compute distances between classes using obtrained GMM.

Define functions for computing distances between components of GMM.

In [15]:
def calc_one_dij(mean_list1, sigma_list1, mean_list2, sigma_list2, i, j):
    return math.sqrt(((mean_list1[i] - mean_list2[j])**2+(sigma_list1[i] - sigma_list2[j])**2).sum())

In [16]:
def calc_dij(mean_list1, sigma_list1, mean_list2, sigma_list2):
    return np.array([calc_one_dij(mean_list1, sigma_list1, mean_list2, sigma_list2, i, j)
              for i in range(num_components)
              for j in range(num_components)]).reshape(num_components, num_components)

Here we use linear programming to obtrain distances.

Assume $w_{ij}$ is ordered like $w_{00}, w_{01}, w_{02}, w_{03}, ... w_{024}, w_{10}, w_{11}, ...$

In [17]:
from scipy.optimize import linprog

$\sum_i  w_{ij} = \beta_j$.

In [18]:
def sum_i_coeff(target_j):
    return [float(j == target_j) for i in range(num_components) for j in range(num_components)]

In [19]:
coeff_i_sums = [sum_i_coeff(j) for j in range(num_components)]

$\sum_j w_{ij} = \alpha_i$.

In [20]:
def sum_j_coeff(target_i):
    return [float(i == target_i) for i in range(num_components) for j in range(num_components)]

In [21]:
coeff_j_sums = [sum_j_coeff(i) for i in range(num_components)]

Take summation of these quantities. 

In [22]:
A_eq = coeff_i_sums+coeff_j_sums

Define a function that computes distances between classes.

In [23]:
def calc_distance(midx1, midx2):
    alpha = gmm_weights[midx1]
    beta = gmm_weights[midx2]
    mus1, sigs1 = gmm_stats[midx1]
    mus2, sigs2 = gmm_stats[midx2]
    dij =  calc_dij(mus1, sigs1, mus2, sigs2)

    b_eq = np.concatenate((beta, alpha))
    c = dij.reshape(num_components*num_components)
    res = linprog(c, A_eq=A_eq, b_eq=b_eq,
              options={"disp": False})
    return res.fun, res

Check the function.

In [32]:
dist, _ = calc_distance(3, 5)
dist

13395.58948641049

In [33]:
dist, _ = calc_distance(5, 3)
dist

13395.58948641049

Define a function that computes all of combinations of classes.

In [34]:
def calc_all_distances():
    catnum = len(cats)
    dists = np.zeros((catnum, catnum))
    for i in range(catnum):
        for j in range(catnum):
            if i != j :
                dists[i, j], _ = calc_distance(i, j)
    return dists

In [35]:
%%time
Dps = calc_all_distances()

CPU times: user 1min 47s, sys: 4 ms, total: 1min 47s
Wall time: 1min 47s


In [36]:
distdict = {cat1:[Dps[idx1, idx2] for idx2 in range(len(cats))] for idx1, cat1 in enumerate(cats)}

In [37]:
distdf = pd.DataFrame(distdict)

In [38]:
distdf.index = cats

In [44]:
distdf

,bay,beach,birds,boeing,buildings,city,clouds,f-16,face,helicopter,mountain,ocean,ships,sky,sunrise,sunset
bay,0.000000,6588.132327,7191.791020,8742.313416,9813.075542,8678.661169,9547.271762,9124.422209,11249.360091,7736.787644,6950.625571,7269.991418,7983.156091,13553.036778,10122.609485,8842.364538
beach,6588.132327,0.000000,7013.673433,8058.638931,10428.612828,9718.150779,8853.541685,8470.443863,11144.968431,7737.724702,6908.724801,8069.756136,8215.840725,12997.335254,9672.171781,8008.298275
birds,7191.791020,7013.673433,0.000000,6489.982225,11193.529932,10679.967999,6680.630874,6489.781659,9025.691628,5656.062784,7117.078511,8898.890890,7520.067444,10693.746482,7828.975030,6666.279988
boeing,8742.313416,8058.638931,6489.982225,0.000000,13738.395314,13395.589486,3525.278568,3438.058438,8123.424867,4917.535086,8323.152387,10683.425380,8563.604679,7467.246789,6027.745841,5365.497668
buildings,9813.075542,10428.612828,11193.529932,13738.395314,0.000000,9623.520669,14415.805486,13867.147478,15108.090130,12369.522756,10634.257474,10152.265733,10921.797552,18221.522006,14527.504012,13017.452170
city,8678.661169,9718.150779,10679.967999,13395.589486,9623.520669,0.000000,14117.528707,13526.248699,14637.481938,11876.476961,9584.863122,8576.396335,10754.561406,17848.555513,14070.365893,12701.345673
clouds,9547.271762,8853.541685,6680.630874,3525.278568,14415.805486,14117.528707,0.000000,3421.153034,8118.116423,5067.434105,8820.950604,11351.753019,9305.763306,6609.064667,6286.898351,5447.433198
f-16,9124.422209,8470.443863,6489.781659,3438.058438,13867.147478,13526.248699,3421.153034,0.000000,7767.589927,4682.250712,8644.150951,10834.365404,8699.688065,7161.422905,5885.224257,5253.432736
face,11249.360091,11144.968431,9025.691628,8123.424867,15108.090130,14637.481938,8118.116423,7767.589927,0.000000,7848.839349,10736.670594,12605.974421,11571.865834,11476.063827,10408.622751,8389.830468
helicopter,7736.787644,7737.724702,5656.062784,4917.535086,12369.522756,11876.476961,5067.434105,4682.250712,7848.839349,0.000000,7603.952902,9487.192423,7506.198536,8964.521617,6591.735071,5634.905014


Save the results as pickle.

In [45]:
distdf.to_pickle("results/GMMDisttances.dat")

## Show distances for each target class

In [46]:
# def show_cat_dist(catidx):
#     print(cats[catidx])
#     print(distdf[cats[catidx]].sort_values())

In [47]:
# [show_cat_dist(i) for i in range(len(cats))]